<a href="https://colab.research.google.com/github/VickyCarlson/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-11 00:36:15--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  1.69MB/s    in 0.6s    

2022-01-11 00:36:16 (1.69 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
url = "https://bookrevs.s3.us-west-2.amazonaws.com/vine_table.csv"
spark.sparkContext.addFile(url)
vine_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("vine_table.csv"), sep=",", header=True, inferSchema=True)
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1C5OK9AND7PRP|          5|            0|          0|   N|                Y|
|R333RNBQMPVUFT|          5|            0|          0|   N|                Y|
|R2A2K0GTNSKXM1|          5|            0|          0|   N|                N|
|R365LAQ9REV876|          3|            0|          0|   N|                Y|
|R294OF3SNH6SWZ|          4|            0|          0|   N|                Y|
|R2OMRH58WYEEP9|          5|            0|          0|   N|                N|
| RBFXYUJZIH5JR|          5|            0|          0|   N|                Y|
|R2K2RSBNV42HU4|          5|            0|          0|   N|                N|
|R2CLRFFJ5HJSU3|          4|            0|          0|   N|                N|
| RS1O7V45AADDO|          4|            0|          0|   N|     

In [7]:
vine_total_ct = vine_df.count()
vine_total_ct

12520722

In [8]:
#Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
vine_total_df = df.filter("total_votes>= 20")
vine_total_df.show()
vine_total_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2HQLKREFKG25D|          5|           21|         21|   N|                Y|
| RWIL1XU6YM0U4|          5|           20|         20|   N|                N|
|R28SB7Q1WBJRFM|          5|           25|         27|   N|                N|
|R3PVKAWMZW55U8|          5|          106|        119|   N|                Y|
|R21JS1HIQV1H7W|          3|           17|         22|   N|                Y|
|R2IBPJ9OIDSZHW|          3|            0|         37|   N|                Y|
|R3SR8MPHQATFCJ|          1|            8|         20|   N|                Y|
|R245IE9DATP3ZL|          3|           15|         23|   N|                Y|
| RMK55ZA87SPQX|          1|           22|         24|   N|                Y|
| RB1Y3RO4J4854|          3|           23|         30|   N|     

70726

In [12]:
# Filter the new DataFrame create a new DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
vine_helpful_votes_df = vine_total_df.filter("(helpful_votes/total_votes)>=0.5")
vine_helpful_votes_df.show()
vine_helpful_votes_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2HQLKREFKG25D|          5|           21|         21|   N|                Y|
| RWIL1XU6YM0U4|          5|           20|         20|   N|                N|
|R28SB7Q1WBJRFM|          5|           25|         27|   N|                N|
|R3PVKAWMZW55U8|          5|          106|        119|   N|                Y|
|R21JS1HIQV1H7W|          3|           17|         22|   N|                Y|
|R245IE9DATP3ZL|          3|           15|         23|   N|                Y|
| RMK55ZA87SPQX|          1|           22|         24|   N|                Y|
| RB1Y3RO4J4854|          3|           23|         30|   N|                Y|
|R23P75H9VX5RLM|          5|           32|         36|   N|                Y|
|R3JCH5P63TIQ4E|          5|           19|         20|   N|     

54092

In [10]:
vine_yes_df = vine_helpful_votes_df.filter("vine=='Y'")
vine_yes_df.show()
vine_yes_df.count()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



0

In [11]:
vine_no_df = vine_helpful_votes_df.filter("vine=='N'")
vine_no_df.show()
vine_no_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2HQLKREFKG25D|          5|           21|         21|   N|                Y|
| RWIL1XU6YM0U4|          5|           20|         20|   N|                N|
|R28SB7Q1WBJRFM|          5|           25|         27|   N|                N|
|R3PVKAWMZW55U8|          5|          106|        119|   N|                Y|
|R21JS1HIQV1H7W|          3|           17|         22|   N|                Y|
|R245IE9DATP3ZL|          3|           15|         23|   N|                Y|
| RMK55ZA87SPQX|          1|           22|         24|   N|                Y|
| RB1Y3RO4J4854|          3|           23|         30|   N|                Y|
|R23P75H9VX5RLM|          5|           32|         36|   N|                Y|
|R3JCH5P63TIQ4E|          5|           19|         20|   N|     

54092

In [13]:
vine_no_df_ct = vine_no_df.count()
vine_no_df_ct

54092

In [14]:
vine_helpful_votes_df_ct= vine_helpful_votes_df.count()
vine_helpful_votes_df_ct

54092

In [15]:
total_helpful_reviews_ct = vine_helpful_votes_df_ct
total_helpful_reviews_ct

54092

In [20]:
Five_Star_All_df = vine_helpful_votes_df.filter("star_rating==5.0")
Five_Star_All_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2HQLKREFKG25D|          5|           21|         21|   N|                Y|
| RWIL1XU6YM0U4|          5|           20|         20|   N|                N|
|R28SB7Q1WBJRFM|          5|           25|         27|   N|                N|
|R3PVKAWMZW55U8|          5|          106|        119|   N|                Y|
|R23P75H9VX5RLM|          5|           32|         36|   N|                Y|
|R3JCH5P63TIQ4E|          5|           19|         20|   N|                N|
| RE754WU5TGN9E|          5|           39|         44|   N|                Y|
|  RVNRTIVFKBBI|          5|           23|         23|   N|                N|
|R26GCFJG6427T4|          5|           29|         29|   N|                Y|
|R31X4TF63XSPKJ|          5|           30|         35|   N|     

In [21]:
Five_Star_All_df_ct = Five_Star_All_df.count()
Five_Star_All_df_ct

20918

In [22]:
Five_Star_No_df = Five_Star_All_df.filter("vine=='N'")
Five_Star_No_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2HQLKREFKG25D|          5|           21|         21|   N|                Y|
| RWIL1XU6YM0U4|          5|           20|         20|   N|                N|
|R28SB7Q1WBJRFM|          5|           25|         27|   N|                N|
|R3PVKAWMZW55U8|          5|          106|        119|   N|                Y|
|R23P75H9VX5RLM|          5|           32|         36|   N|                Y|
|R3JCH5P63TIQ4E|          5|           19|         20|   N|                N|
| RE754WU5TGN9E|          5|           39|         44|   N|                Y|
|  RVNRTIVFKBBI|          5|           23|         23|   N|                N|
|R26GCFJG6427T4|          5|           29|         29|   N|                Y|
|R31X4TF63XSPKJ|          5|           30|         35|   N|     

In [23]:
Five_Star_No_df_ct = Five_Star_No_df.count()
Five_Star_No_df_ct

20918

In [24]:
Five_Star_Yes_df = Five_Star_All_df.filter("vine=='Y'")
Five_Star_Yes_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [25]:
Five_Star_Yes_df_ct = Five_Star_Yes_df.count()
Five_Star_Yes_df_ct

0

In [26]:
Percent_Yes_5Stars = (Five_Star_Yes_df_ct/Five_Star_All_df_ct)*100
Percent_Yes_5Stars

0.0

In [27]:
Percent_No_5Stars = (Five_Star_No_df_ct/Five_Star_All_df_ct)*100
Percent_No_5Stars

100.0

In [29]:
Percent_Yes_5Stars_ALL = (Five_Star_Yes_df_ct/vine_total_ct)*100
Percent_Yes_5Stars_ALL

0.0

In [30]:
Percent_No_5Stars_ALL = (Five_Star_No_df_ct/vine_total_ct)*100
Percent_No_5Stars_ALL

0.16706704293889763